# Exploratory Data Analysis
## SPY Options Data from WRDS OptionMetrics

In [ ]:
import os
import sys
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

repo_path = os.getenv("REPO_PATH") or os.path.abspath("..")
sys.path.append(os.path.join(repo_path, "src"))

from data_loader import (
    load_raw_data,
    get_spot_price,
    prepare_option_data,
    split_data
)
from preprocessing import (
    clean_data,
    compute_implied_volatilities,
    normalize_features,
    create_model_dataset,
    add_additional_features
)
from visualize import (
    plot_volatility_smile,
    plot_volatility_surface_3d,
    plot_volatility_heatmap,
    plot_term_structure,
    check_calendar_arbitrage,
    check_butterfly_arbitrage,
    plot_arbitrage_analysis,
    plot_bid_ask_spread_analysis,
    summarize_data_characteristics
)
from utils import set_seed

set_seed(42)

%matplotlib inline
plt.style.use("seaborn-v0_8-darkgrid")
sns.set_context("talk", font_scale=0.9)
pd.set_option("display.max_columns", 50)


## 1. Load Raw Data

In [ ]:
raw_data_path = os.getenv("RAW_DATA_PATH") or os.path.join(repo_path, "data", "raw")
ticker_name = os.getenv("TICKER_NAME", "SPY")
date = os.getenv("TRADE_DATE", "2023-01-03")

if not os.path.isdir(raw_data_path):
    raise FileNotFoundError(f"Raw data path not found: {raw_data_path}")

print("Loading raw data...")
data = load_raw_data(raw_data_path, ticker_name, date)

if len(data) == 0:
    raise ValueError("No parquet files were loaded; check RAW_DATA_PATH and filename patterns.")

print("
Loaded datasets:")
for key, df in data.items():
    print(f"  {key}: {len(df)} rows, {len(df.columns)} columns")


## 2. Examine Option Price Data

In [ ]:
op_df = data.get('option_price')
if op_df is None:
    raise KeyError("option_price parquet missing; expected load_raw_data to find *_option_price.parquet")

print(f"Option Price Data Shape: {op_df.shape}")
print(f"
Columns: {list(op_df.columns)}")
print("
First few rows:")
op_df.head()


In [ ]:
print("Summary Statistics:")
op_df[['strike_price', 'best_bid', 'best_offer', 'volume', 'open_interest', 'impl_volatility']].describe()


In [ ]:
print("
Option Type Distribution:")
print(op_df['cp_flag'].value_counts())


## 3. Market Parameters

In [ ]:
security_df = data.get('security_price')
zero_curve_df = data.get('zero_curve')
if security_df is None or zero_curve_df is None:
    raise KeyError("security_price and zero_curve parquet files are required for downstream processing")

spot_price = get_spot_price(security_df)
print(f"SPY Spot Price: ${spot_price:.2f}")

print("
Zero Curve:")
zero_curve_df.head()


## 4. Prepare Option Data

In [ ]:
print("Preparing option data...")
prepared_df = prepare_option_data(
    op_df,
    spot_price,
    date,
    zero_curve_df,
    data.get('distr_proj', None)
)

print(f"Prepared data shape: {prepared_df.shape}")
print("
New columns added:", [col for col in prepared_df.columns if col not in op_df.columns])
prepared_df.head()


## 5. Clean Data

In [ ]:
print(f"Before cleaning: {len(prepared_df)} options")
cleaned_df = clean_data(prepared_df)
print(f"After cleaning: {len(cleaned_df)} options")
print(f"Removed: {len(prepared_df) - len(cleaned_df)} options ({100*(len(prepared_df) - len(cleaned_df))/len(prepared_df):.1f}%)")


## 6. Compute Implied Volatilities

In [ ]:
print("Computing implied volatilities...")
iv_df = compute_implied_volatilities(cleaned_df)
print(f"Successfully computed IV for {len(iv_df)} options")

if 'impl_volatility' in iv_df.columns:
    valid_market_iv = iv_df[iv_df['impl_volatility'].notna()]
    if len(valid_market_iv) > 0:
        print("
IV Comparison (computed vs market):")
        print(f"  Mean difference: {(valid_market_iv['computed_iv'] - valid_market_iv['impl_volatility']).mean():.4f}")
        print(f"  Median difference: {(valid_market_iv['computed_iv'] - valid_market_iv['impl_volatility']).median():.4f}")


In [ ]:
iv_df = add_additional_features(iv_df)
print(f"Final dataset shape: {iv_df.shape}")
print("
Summary statistics for computed IV:")
iv_df['computed_iv'].describe()


### 6a. Dataset Characteristics

In [ ]:
summary = summarize_data_characteristics(iv_df)
print("Key ranges and counts:")
summary_df = pd.Series(summary, name="value")
summary_df

## 7. Visualize Distributions

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(iv_df['moneyness'], bins=50, alpha=0.7, edgecolor='black')
axes[0].axvline(1.0, color='red', linestyle='--', label='ATM')
axes[0].set_xlabel('Moneyness (K/S)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Moneyness')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].hist(iv_df['T'], bins=50, alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Time to Maturity (years)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Time to Maturity')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 8. Smiles and Surfaces

In [ ]:
maturity_buckets = [30, 60, 90, 180]
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
plot_volatility_smile(iv_df, maturity_buckets, option_type='C', ax=axes[0])
plot_volatility_smile(iv_df, maturity_buckets, option_type='P', ax=axes[1])
plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(10, 6))
plot_term_structure(iv_df, moneyness_levels=[0.9, 1.0, 1.1], ax=ax)
plt.tight_layout()
plt.show()


In [ ]:
fig = plot_volatility_surface_3d(iv_df, iv_col='computed_iv', interpolate=True, grid_size=50)
plt.show()

fig = plot_volatility_heatmap(iv_df, iv_col='computed_iv', grid_size=(40, 40))
plt.show()


## 9. Arbitrage and Market Quality Checks

In [ ]:
calendar_violations = check_calendar_arbitrage(iv_df, iv_col='computed_iv', moneyness_tolerance=0.02)
butterfly_violations = check_butterfly_arbitrage(iv_df, price_col='mid_price', maturity_tolerance=2)

print(f"Calendar arbitrage violations: {len(calendar_violations)}")
print(f"Butterfly arbitrage violations: {len(butterfly_violations)}")

if len(calendar_violations) > 0:
    display(calendar_violations.head())
if len(butterfly_violations) > 0:
    display(butterfly_violations.head())

fig = plot_arbitrage_analysis(calendar_violations, butterfly_violations)
plt.show()

fig = plot_bid_ask_spread_analysis(iv_df)
plt.show()


## 10. Split and Normalize Data

In [ ]:
train_df, test_df = split_data(iv_df, train_ratio=0.8, random_state=42)
print(f"Training set: {len(train_df)} samples")
print(f"Test set: {len(test_df)} samples")


In [ ]:
input_features = ['log_moneyness', 'T']
target = 'computed_iv'

X_train, y_train, norm_params = create_model_dataset(
    train_df,
    input_features=input_features,
    target=target,
    normalize=True
)

X_test, y_test, _ = create_model_dataset(
    test_df,
    input_features=input_features,
    target=target,
    normalize=False
)

test_df_normalized = test_df.copy()
for feature in input_features:
    mean = norm_params[feature]['mean']
    std = norm_params[feature]['std'] + 1e-8
    test_df_normalized[feature] = (test_df[feature] - mean) / std
X_test = test_df_normalized[input_features].values

print(f"
Training set shape: X={X_train.shape}, y={y_train.shape}")
print(f"Test set shape: X={X_test.shape}, y={y_test.shape}")
print("
Normalization parameters:")
norm_params


## 11. Save Processed Data

In [ ]:
processed_path = os.path.join(repo_path, "data", "processed")
os.makedirs(processed_path, exist_ok=True)

train_df.to_parquet(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_train.parquet"))
test_df.to_parquet(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_test.parquet"))

np.save(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_X_train.npy"), X_train)
np.save(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_y_train.npy"), y_train)
np.save(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_X_test.npy"), X_test)
np.save(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_y_test.npy"), y_test)

import json
with open(os.path.join(processed_path, f"{ticker_name}_{date.replace('-', '')}_norm_params.json"), 'w') as f:
    json.dump(norm_params, f, indent=4)

print(f"Processed data saved to: {processed_path}")


## Summary

- Loaded WRDS OptionMetrics raw parquet files and extracted spot/zero-curve context
- Cleaned invalid quotes and computed implied vols using Black–Scholes
- Profiled ranges (moneyness, maturities) and visualized smiles, surfaces, and term structures
- Checked calendar/butterfly arbitrage and bid-ask quality for anomalies
- Normalized features, split train/test, and saved processed artifacts for downstream modeling